This notebook uses word embeddings to implement the method of [Turney and Littman (2003)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.6425&rep=rep1&type=pdf) for calculating the semantic orientation of a term defined by proximity to other terms in two polar dictionaries.

In [ ]:
import re, operator
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath

In [ ]:
turney_littman_positive=set(["good", "nice", "excellent", "positive", "fortunate", "correct", "superior"])
turney_littman_negative=set(["bad", "nasty", "poor", "negative", "unfortunate", "wrong", "inferior"])

Download the Glove vectors [glove.42B.300d.50K.txt](https://drive.google.com/file/d/1n1jt0UIdI3CD26cY1EIeks39XH5S8O8M/view?usp=sharing) and [glove.twitter.27B.100d.50K.txt](https://drive.google.com/file/d/1Tk4S5u6mwwZwEd5H7bimNXzHnbqWw7_y/view?usp=sharing) and store them in the data/ directory.  These are word vectors from [Glove](https://nlp.stanford.edu/projects/glove/), with the most frequent 50K words from each source.

In [ ]:
def read_glove(filename):

    # First we have to convert the Glove format into w2v format; this creates a new file

    glove_in_w2v_format="%s.w2v" % filename
    _ = glove2word2vec(filename, glove_in_w2v_format)
    
    glove = KeyedVectors.load_word2vec_format(glove_in_w2v_format, binary=False)
    return glove

In [ ]:
common_crawl_vectors=read_glove("../data/glove.42B.300d.50K.txt")

In [ ]:
twitter_vectors=read_glove("../data/glove.twitter.27B.100d.50K.txt")

Q1: Implement the [Turney and Littman (2003)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.6425&rep=rep1&type=pdf) method for calculating the semantic orientation of a term using its neighbors (you can find the method on p. 6).  Use word embeddings to calculate $A$(word1, word2) -- the measure of association between word 1 and word 2.  The arguments to `semantic_orientation` function are a set of word embeddings, the query term, and a positive and negative dictionary; it should return the semantic orientation for the query term (a real value).

In [ ]:
def semantic_orientation(embeddings, word, positive_dictionary, negative_dictionary):
    SO=0.
    for query in positive_dictionary:
        similarity=embeddings.similarity(word, query)
        SO+=similarity
    for query in negative_dictionary:
        similarity=embeddings.similarity(word, query)
        SO-=similarity
    return SO

In [ ]:
def find_all_semantic_orientation(glove, positive, negative):
    scores=defaultdict(float)
    for word in glove.vocab:
        scores[word]=semantic_orientation(glove, word, positive, negative)
    
    sorted_x = sorted(scores.items(), key=operator.itemgetter(1))
        
    for k,v in reversed(sorted_x[-25:]):
        print("%.3f\t%s" % (v,k))
    print()

    for k,v in sorted_x[:25]:
        print("%.3f\t%s" % (v,k))

In [ ]:
find_all_semantic_orientation(common_crawl_vectors, turney_littman_positive, turney_littman_negative)

In [ ]:
find_all_semantic_orientation(twitter_vectors, turney_littman_positive, turney_littman_negative)

Q2: In homework 4.classificadtion/FeatureExploration, we created two dictionaries to use for binary text classification.  For the binary classification problem you have been working on, create two new dictionaries containing at least 10 terms each that you think will help in discriminating between the two classes.

Execute this method on that pair of dictionaries to discover new terms to fill it.  How many of the top 10 terms are you able to select as appropriate for the dictionaries?

In [ ]:
positive_class_dictionary=set([""])
negative_class_dictionary=set([""])

In [ ]:
find_all_semantic_orientation(common_crawl_vectors, positive_class_dictionary, negative_class_dictionary)